In [2]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
from torchvision.datasets import ImageFolder 
from tqdm.auto import tqdm
import os
import pandas as pd
from torchsummary import summary
from collections import OrderedDict 

In [14]:
# Check if CUDA is available
use_cuda = torch.cuda.is_available()
if not use_cuda:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [9]:
# Define transform and import dataset
transforms1 = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.CenterCrop((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transforms2 = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.CenterCrop((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.RandomHorizontalFlip(p=0.5)
])

"""

full_dataset = ImageFolder('../data/raw/Taiwanese-Food-101', transform=transforms1)

# Define generator and set split size
generator1 = torch.Generator().manual_seed(38)
train_size = int(0.7 * len(full_dataset))
test_size = val_size = (len(full_dataset) - train_size)//2

# Split dataset to train, test, and validation
train_dataset, test_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size, val_size], generator = generator1)

train_dataset.dataset.transform = transforms2

"""

train_dataset = torch.load('../data/processed/Taiwanese-Food-101/train/train_dataset.pt')
val_dataset = torch.load("../data/processed/Taiwanese-Food-101/val/val_dataset.pt")
test_dataset = torch.load("../data/processed/Taiwanese-Food-101/test/test_dataset.pt")

# Define hypterparameters
batch_size = 32
LR = 0.001
criterion = nn.CrossEntropyLoss()

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

class_names = test_dataset.dataset.classes

In [ ]:
# Load the pretrained model
state_dict = torch.load('../models/densenet161_unfreeze2_sch')

model = models.densenet161(weights = "DEFAULT")

# Freeze the parameters of the pre-trained model to avoid updating them during training
for param in model.parameters():
    param.requires_grad = False
    

for param in list(model.features.denseblock4.parameters()) + list(model.features.norm5.parameters()) + list(model.features.denseblock3.parameters()):
    param.requires_grad = True

# Replace the classifier with your custom classifier
model.classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(2208, 500)),  # Adjusted input size to match DenseNet-161 output
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 101)),  # Adjusted output size to match the number of classes (101 in this case)
                          ('output', nn.LogSoftmax(dim=1))
                          ]))


# Verify model structure and 
_ = summary(model.cuda(), (3,224,224))

model.load_state_dict(state_dict)

In [20]:
def test(loader, model, criterion, use_cuda, class_names):
    test_loss = 0.
    correct_top1 = 0
    correct_top5 = 0
    total = 0
    all_targets = []
    all_predictions = []
    
    model.eval()
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(tqdm(loader, desc='Testing')):
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            
            output = model(data)
            loss = criterion(output, target)
            
            test_loss += loss.item() * data.size(0)
            
            _, pred_top1 = output.topk(1, dim=1)
            correct_top1 += torch.sum(pred_top1.view(-1) == target).item()
            
            _, pred_top5 = output.topk(5, dim=1)
            correct_top5 += torch.sum(pred_top5 == target.view(-1, 1).expand_as(pred_top5)).item()
            
            total += target.size(0)
            
            all_targets.extend(target.cpu().numpy())
            all_predictions.extend(pred_top1.view(-1).cpu().numpy())
    
    test_loss /= total
    top1_accuracy = 100. * correct_top1 / total
    top5_accuracy = 100. * correct_top5 / total
    
    print(f'Test Loss: {test_loss:.6f}')
    print(f'Test Top-1 Accuracy: {top1_accuracy:.2f}% ({correct_top1}/{total})')
    print(f'Test Top-5 Accuracy: {top5_accuracy:.2f}% ({correct_top5}/{total})')
    
    # Create DataFrame
    df = pd.DataFrame({
        'Sample ID': range(1, total + 1),
        'Ground Truth': [class_names[t] for t in all_targets],
        'Predicted Label': [class_names[p] for p in all_predictions]
    })
    
    return df, test_loss, top1_accuracy, top5_accuracy

In [21]:
if use_cuda:
    model.cuda()

df, test_loss, top1_accuracy, top5_accuracy = test(test_loader, model, criterion, use_cuda, class_names)

Testing:   0%|          | 0/95 [00:00<?, ?it/s]

Test Loss: 0.307990
Test Top-1 Accuracy: 91.98% (2787/3030)
Test Top-5 Accuracy: 98.91% (2997/3030)
